In [ ]:
!pip install arxiv

In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
!pip install wikipedia

  Using cached wikipedia-1.4.0.tar.gz (27 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11706 sha256=95297ebaf495ce8abedf0f2c499c08c1a5d67b3b0e94e5ccfdaa47e261015e8d
  Stored in directory: c:\users\techlanders\appdata\local\pip\cache\wheels\63\47\7c\a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


In [16]:
# https://python.langchain.com/v0.1/docs/integrations/tools/wikipedia/

api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [17]:
#first tool
wiki.name

'wikipedia'

In [8]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [9]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# https://python.langchain.com/v0.1/docs/integrations/document_loaders/web_base/

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
retriever=vectordb.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000243F50858B0>)

In [10]:
# https://python.langchain.com/v0.1/docs/modules/agents/quick_start/#retriever

from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [15]:
#second tool
retriever_tool.name

'langsmith_search'

In [19]:
!pip install arxiv

  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/81.1 kB ? eta -:--:--
   ---------------------------------------- 81.1/81.1 kB 2.2 MB/s eta 0:00:00
Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6060 sha256=63dc5af2ae64d12b6d53ea9c96ec27d7707746a10fd972ddeff3ab3f5c1f712a
  Stored in directory: c:\users\techlanders\appdata\local\pip\cache\wheels\03\f5\1a\23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k
  Attempting uninstall: requests
    Found existing installation: requests 2.32.2
    Uninstalling requests-2.32.2:
      Successfully uninstalled requests-2.32.2


In [20]:
## Arxiv Tool (third tool)
# https://python.langchain.com/v0.1/docs/integrations/tools/arxiv/

from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [21]:
tools=[wiki,arxiv,retriever_tool]

In [22]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\techlanders\\test\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000243EC1A0C20>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vecto

In [23]:
from dotenv import load_dotenv

load_dotenv()
import os
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [25]:
!pip install langchainhub

In [26]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [27]:
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [28]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [29]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key​To create an API key head to the Settings page. Then click Create API Key.3. Set up your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first trace​We prov

{'input': 'Tell me about Langsmith',
 'output': 'LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. LangSmith works on its own and does not require the use of LangChain. \n\nTo get started with LangSmith, you can follow these steps:\n1. Install LangSmith using Python or TypeScript.\n2. Create an API key by heading to the Settings page and clicking on Create API Key.\n3. Set up your environment by exporting necessary variables like LANGCHAIN_TRACING_V2 and LANGCHAIN_API_KEY.\n4. Log your first trace by using the provided methods to log traces to LangSmith.\n\nLangSmith provides tools for creating datasets, examples, evaluators, and running evaluations on your applications. You can learn more about evaluation in the how-to guides provided by LangSmith.\n\nFor more detailed information, you can visit the LangSmith documentation on their w

In [30]:
agent_executor.invoke({"input":"What's the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
alloThe paper with the identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch. It discusses the study of graphs on lattice points where the edges are derived from a finite set of allocations.

> Finished chain.


{'input': "What's the paper 1605.08386 about?",
 'output': 'The paper with the identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch. It discusses the study of graphs on lattice points where the edges are derived from a finite set of allocations.'}

In [31]:
agent_executor.invoke({"input":"What's machine learning?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and genMachine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generate insights or predictions. It involves creating models that can automatically learn and improve from experience without being explicitly programmed.

> Finished chain.


{'input': "What's machine learning?",
 'output': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generate insights or predictions. It involves creating models that can automatically learn and improve from experience without being explicitly programmed.'}